# Recommender

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
%matplotlib inline


column_names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep = '\t', names = column_names)
movies_titles = pd.read_csv('Movie_Id_Titles')

In [2]:
df = pd.merge(df,movies_titles, on= 'item_id')
df.head()

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)
2,79,50,4,891271545,Star Wars (1977)
3,2,50,5,888552084,Star Wars (1977)
4,8,50,5,879362124,Star Wars (1977)


In [3]:
df_ratings = pd.DataFrame(df.groupby('title')['rating'].mean())
df_ratings['num_ratings'] = pd.DataFrame(df.groupby('title')['rating'].count())
df_ratings.head()

,rating,num_ratings
title,,
'Til There Was You (1997),2.333333,9
1-900 (1994),2.600000,5
101 Dalmatians (1996),2.908257,109
12 Angry Men (1957),4.344000,125
187 (1997),3.024390,41


In [4]:
movie_pivot = df.pivot_table(index = 'user_id', columns='title',values = 'rating')
movie_pivot.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
similar_to_star = movie_pivot.corrwith(movie_pivot['Star Wars (1977)'])
similar_to_liar = movie_pivot.corrwith(movie_pivot['Liar Liar (1997)'])
similar_to_star

/Users/me/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/Users/me/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


title
'Til There Was You (1997)                0.872872
1-900 (1994)                            -0.645497
101 Dalmatians (1996)                    0.211132
12 Angry Men (1957)                      0.184289
187 (1997)                               0.027398
                                           ...   
Young Guns II (1990)                     0.228615
Young Poisoner's Handbook, The (1995)   -0.007374
Zeus and Roxanne (1997)                  0.818182
unknown                                  0.723123
Á köldum klaka (Cold Fever) (1994)            NaN
Length: 1664, dtype: float64

In [6]:
df_corr_starwars = pd.DataFrame(similar_to_star, columns = ['Correlation'])
df_corr_starwars.dropna(inplace = True)
df_corr_starwars.sort_values(by ='Correlation',ascending=False).head()

,Correlation
title,
Commandments (1997),1.0
Cosi (1996),1.0
No Escape (1994),1.0
Stripes (1981),1.0
Man of the Year (1995),1.0


In [7]:
df_corr_starwars = df_corr_starwars.join(df_ratings['num_ratings'])
df_corr_starwars.head()

,Correlation,num_ratings
title,,
'Til There Was You (1997),0.872872,9
1-900 (1994),-0.645497,5
101 Dalmatians (1996),0.211132,109
12 Angry Men (1957),0.184289,125
187 (1997),0.027398,41


In [8]:
df_corr_starwars[df_corr_starwars['num_ratings']>100].sort_values(by='Correlation',ascending=False)

,Correlation,num_ratings
title,,
Star Wars (1977),1.000000,584
"Empire Strikes Back, The (1980)",0.748353,368
Return of the Jedi (1983),0.672556,507
Raiders of the Lost Ark (1981),0.536117,420
Austin Powers: International Man of Mystery (1997),0.377433,130
...,...,...
"Edge, The (1997)",-0.127167,113
As Good As It Gets (1997),-0.130466,112
Crash (1996),-0.148507,128


In [9]:
df_similar_to_liar = pd.DataFrame(similar_to_liar,columns = ['Correlation'])
df_similar_to_liar.dropna(inplace = True)
df_similar_to_liar = df_similar_to_liar.join(df_ratings['num_ratings'])
df_similar_to_liar.sort_values(by ='Correlation',ascending=False).head()

,Correlation,num_ratings
title,,
"Bonheur, Le (1965)",1.0,4
For Ever Mozart (1996),1.0,3
Crossfire (1947),1.0,4
Johnny 100 Pesos (1993),1.0,2
Moonlight and Valentino (1995),1.0,7


In [10]:
df_similar_to_liar[df_similar_to_liar['num_ratings']>100].sort_values(by='Correlation',ascending=False)

,Correlation,num_ratings
title,,
Liar Liar (1997),1.000000,485
Batman Forever (1995),0.516968,114
"Mask, The (1994)",0.484650,129
Down Periscope (1996),0.472681,101
Con Air (1997),0.469828,137
...,...,...
Hoop Dreams (1994),-0.184503,117
Ed Wood (1994),-0.199481,133
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963),-0.238092,194


In [12]:
#a = 'Contact (1997)'

df_ratings = pd.DataFrame(df.groupby('title')['rating'].mean())
df_ratings['num_ratings'] = pd.DataFrame(df.groupby('title')['rating'].count())

movie_pivot = df.pivot_table(index = 'user_id', columns='title',values = 'rating')


similar_to_liar = movie_pivot.corrwith(movie_pivot['Liar Liar (1997)'])

df_similar_to_liar = pd.DataFrame(similar_to_liar,columns = ['Correlation'])
df_similar_to_liar.dropna(inplace = True)
df_similar_to_liar = df_similar_to_liar.join(df_ratings['num_ratings'])
df_similar_to_liar[df_similar_to_liar['num_ratings']>100].sort_values(by='Correlation',ascending=False).head()

/Users/me/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/Users/me/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,Correlation,num_ratings
title,,
Liar Liar (1997),1.000000,485
Batman Forever (1995),0.516968,114
"Mask, The (1994)",0.484650,129
Down Periscope (1996),0.472681,101
Con Air (1997),0.469828,137


In [17]:
#X = 'Contact (1997)'
X = '12 Angry Men (1957)'

df_ratings = pd.DataFrame(df.groupby('title')['rating'].mean())
df_ratings['num_ratings'] = pd.DataFrame(df.groupby('title')['rating'].count())
movie_pivot = df.pivot_table(index = 'user_id', columns='title',values = 'rating')

a = movie_pivot.corrwith(movie_pivot[X])
a = pd.DataFrame(a,columns = ['Correlation'])
a.dropna(inplace = True)
a = a.join(df_ratings['num_ratings'])
a[a['num_ratings']>100].sort_values(by='Correlation',ascending=False).head()

/Users/me/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/Users/me/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,Correlation,num_ratings
title,,
12 Angry Men (1957),1.000000,125
Ulee's Gold (1997),0.619544,184
Rear Window (1954),0.570513,209
Seven Years in Tibet (1997),0.549939,155
Clerks (1994),0.528173,148
